In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd 

from textblob import TextBlob
import pyprojroot.here as here

In [3]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd 
import numpy as np
import re

##LDA stuff
import gensim
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

##cleaning stuff
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import gensim.corpora as corpora
import pyLDAvis
import pyLDAvis.gensim
from gensim.models import CoherenceModel

##plotting
from matplotlib import pyplot as plt
from matplotlib import colors as mcolors
import seaborn as sns

from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pyprojroot.here as here

In [4]:
# TODO: modify these list if needed (eg. if you want to load only 1 csv from star3, delete other csvs in star3 list)
star3 = ['cleaned_ibis-sg-bencoolen.csv','cleaned_hotel-boss.csv','cleaned_hotel-G.csv',
           'cleaned_village-hotel-albert-court-by-far-east-hospitality.csv',
           'cleaned_holiday-inn-express-clarke-quay.csv']
star4 = ['cleaned_village-hotel-changi-by-far-east-hospitality.csv',
         'cleaned_park-regis.csv', 'cleaned_grand-mercure-sg-roxy.csv',
         'cleaned_paradox-sg-merchant-court.csv','cleaned_crowne-plaza.csv']
star5 = ['cleaned_fullerton.csv', 'cleaned_parkroyal-collection-marina-bay.csv', 'cleaned_pan-pacific.csv',
          'cleaned_mbs_total.csv', 'cleaned_swissotel-the-stamford.csv']

RAW_FOLDER = "data/processed/"

def combine_csv_to_dataframe(file_names, all_star = False, filterDate = True):
    """
    Combine multiple CSV files into a single DataFrame.

    Parameters:
    file_names (list): List of CSV file names. 
    all_star (bool): whether or not to load all the hotels (False if only want to load 1 type of hotel star). 
    filterData (bool): whether or not to remove all data dated before 2015

    Returns:
    pd.DataFrame: Combined DataFrame.
    """
    combined_df = pd.DataFrame()

    for file_name in file_names:
        file_interim_path = RAW_FOLDER + file_name
        file_path = here(file_interim_path)
        try:
            df = pd.read_csv(file_path)
            if all_star:
                if file_name in star3:
                    df["star"] = 3
                elif file_name in star4:
                    df["star"] = 4
                else:
                    df["star"] = 5
            #print(f"Length of {file_name} is {len(df)}")
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            #print(len(combined_df))
        except FileNotFoundError:
            print(f"File not found: {file_name}")
        except pd.errors.EmptyDataError:
            print(f"Empty or invalid CSV file: {file_name}")
            
    combined_df = combined_df[combined_df['date'] >= '2015-01-01']
                                
    return combined_df
data = combine_csv_to_dataframe(star3+star4+star5, all_star = True, filterDate = True)
#data[['traveller_username','date','travel_type','traveller_total_contributions','traveller_total_helpful_contributions','review_title','review_text','rating']].head(5)
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 68292 entries, 0 to 98107
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   travel_type      31354 non-null  object 
 1   rating           54837 non-null  float64
 2   label            54837 non-null  object 
 3   combined_review  68292 non-null  object 
 4   date             68292 non-null  object 
 5   covid            68292 non-null  object 
 6   is_local         68292 non-null  int64  
 7   stem_review      68292 non-null  object 
 8   lem_review       68292 non-null  object 
 9   cleaned_review   68292 non-null  object 
 10  star             68292 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 6.3+ MB


,travel_type,rating,label,combined_review,date,covid,is_local,stem_review,lem_review,cleaned_review,star
0,couple,4.0,Positive,clean comfortable hotel expensive find decent ...,2023-08-01,PostCovid,0,clean comfort hotel expens find decent hotel e...,clean comfortable hotel expensive find decent ...,clean comfortable hotel expensive find decent ...,3
1,family,5.0,Positive,good hotel great location great place location...,2023-08-01,PostCovid,0,good hotel great locat great place locat great...,good hotel great location great place location...,good hotel great location great place location...,3
2,friends,5.0,Positive,good place decent price good place good price ...,2022-10-01,PostCovid,0,good place decent price good place good price ...,good place decent price good place good price ...,good place decent price good place good price ...,3
3,solo,5.0,Positive,great location great staff ibis neat tidy hote...,2023-08-01,PostCovid,0,great locat great staff ibi neat tidi hotel li...,great location great staff ibis neat tidy hote...,great location great staff ibis neat tidy hote...,3
4,business,4.0,Positive,good budget stay stayed days nice location sev...,2022-08-01,PostCovid,0,good budget stay stay day nice locat seven ele...,good budget stay stay day nice location seven ...,good budget stay stayed days nice location sev...,3


In [5]:
df3 = data.query('star==3')
df4 = data.query('star==4')
df5 = data.query('star==5')

# Assign LDA topics

In [6]:
lda3 = [(0,
  [('check', 0.07754221),
   ('room', 0.038632233),
   ('late', 0.0287011),
   ('us', 0.02297527),
   ('free', 0.021750309),
   ('flight', 0.021106116),
   ('phone', 0.01733447),
   ('hotel', 0.016891772),
   ('even', 0.0135511635),
   ('call', 0.013317952),
   ('day', 0.012166229),
   ('offer', 0.011367533),
   ('use', 0.011115009),
   ('shower', 0.010950918),
   ('avail', 0.010851161),
   ('staff', 0.010670348),
   ('night', 0.010132152),
   ('book', 0.009971127),
   ('love', 0.009783751),
   ('great', 0.009183307)]),
 (1,
  [('breakfast', 0.034278683),
   ('pool', 0.030564915),
   ('hotel', 0.027128836),
   ('room', 0.026081096),
   ('great', 0.02059655),
   ('good', 0.019135173),
   ('small', 0.018040646),
   ('area', 0.017044589),
   ('well', 0.013666795),
   ('roof', 0.012311558),
   ('view', 0.011596671),
   ('holiday', 0.011423616),
   ('night', 0.010018452),
   ('gym', 0.009669362),
   ('stay', 0.009555898),
   ('shower', 0.009173747),
   ('swim', 0.009164433),
   ('nice', 0.0087079005),
   ('new', 0.008501584),
   ('love', 0.008325661)]),
 (2,
  [('hotel', 0.06655728),
   ('room', 0.043477777),
   ('good', 0.04198849),
   ('stay', 0.028294839),
   ('walk', 0.026103849),
   ('great', 0.025056925),
   ('breakfast', 0.020736806),
   ('clean', 0.01768515),
   ('staff', 0.016517181),
   ('nice', 0.016072808),
   ('small', 0.015918192),
   ('restaur', 0.013748351),
   ('station', 0.012482119),
   ('place', 0.01218387),
   ('close', 0.011329482),
   ('comfort', 0.010791186),
   ('well', 0.010505962),
   ('near', 0.010065788),
   ('food', 0.009885922),
   ('also', 0.009489767)]),
 (3,
  [('hotel', 0.06969013),
   ('stay', 0.066417955),
   ('staff', 0.05314026),
   ('good', 0.031697877),
   ('room', 0.030839775),
   ('help', 0.028683912),
   ('great', 0.026970103),
   ('nice', 0.018887969),
   ('thank', 0.016651997),
   ('clean', 0.016002953),
   ('us', 0.015559769),
   ('time', 0.014691115),
   ('would', 0.012003786),
   ('love', 0.011982997),
   ('excel', 0.011374717),
   ('recommend', 0.01110902),
   ('back', 0.01021663),
   ('come', 0.010141319),
   ('place', 0.009733346),
   ('alway', 0.009532075)]),
 (4,
  [('room', 0.05650268),
   ('hotel', 0.02604588),
   ('one', 0.014327315),
   ('small', 0.012325287),
   ('bed', 0.011888246),
   ('get', 0.010148149),
   ('night', 0.010002165),
   ('stay', 0.009731314),
   ('bathroom', 0.009189536),
   ('floor', 0.008303965),
   ('door', 0.0077738706),
   ('time', 0.007364786),
   ('us', 0.007322691),
   ('two', 0.0067702667),
   ('go', 0.0063244477),
   ('clean', 0.0062881834),
   ('book', 0.006263637),
   ('like', 0.005959722),
   ('even', 0.0059387563),
   ('could', 0.0057766265)])]

In [7]:
lda4 = [(0,
  [('club', 0.036925957),
   ('breakfast', 0.025991233),
   ('drink', 0.023070883),
   ('food', 0.020987445),
   ('room', 0.020425169),
   ('us', 0.017971236),
   ('view', 0.016021611),
   ('buffet', 0.015016759),
   ('also', 0.014502782),
   ('even', 0.013426476),
   ('pool', 0.011997616),
   ('birthday', 0.011323739),
   ('bar', 0.010047487),
   ('day', 0.010044382),
   ('floor', 0.010005477),
   ('suit', 0.009804599),
   ('book', 0.0093631195),
   ('dinner', 0.009028102),
   ('offer', 0.008830276),
   ('restaur', 0.008595202)]),
 (1,
  [('room', 0.057112604),
   ('hotel', 0.027170941),
   ('check', 0.021618905),
   ('time', 0.012995237),
   ('stay', 0.012451955),
   ('book', 0.011236942),
   ('get', 0.010194618),
   ('one', 0.009875403),
   ('would', 0.009275325),
   ('good', 0.009056207),
   ('ask', 0.008433318),
   ('staff', 0.008300225),
   ('us', 0.008119011),
   ('bed', 0.0078031574),
   ('work', 0.007563566),
   ('call', 0.007525502),
   ('night', 0.0073206867),
   ('day', 0.007264555),
   ('desk', 0.0071158716),
   ('need', 0.0070403973)]),
 (2,
  [('stay', 0.057838053),
   ('hotel', 0.0555172),
   ('staff', 0.054285772),
   ('great', 0.029434808),
   ('help', 0.025269043),
   ('room', 0.021166027),
   ('excel', 0.017590908),
   ('time', 0.015353893),
   ('alway', 0.014782647),
   ('us', 0.014055845),
   ('thank', 0.013846903),
   ('would', 0.012430652),
   ('good', 0.011135549),
   ('love', 0.011071707),
   ('recommend', 0.01103534),
   ('make', 0.00991873),
   ('well', 0.009439135),
   ('clean', 0.009351089),
   ('back', 0.009302918),
   ('best', 0.009131312)]),
 (3,
  [('hotel', 0.06491728),
   ('airport', 0.053817067),
   ('room', 0.037729844),
   ('stay', 0.035340644),
   ('flight', 0.025825297),
   ('termin', 0.025545131),
   ('night', 0.018983345),
   ('check', 0.0177292),
   ('great', 0.014641249),
   ('one', 0.013097579),
   ('comfort', 0.012926798),
   ('transit', 0.010138017),
   ('good', 0.009881743),
   ('hour', 0.009734009),
   ('get', 0.009669625),
   ('walk', 0.0095247775),
   ('well', 0.009110735),
   ('would', 0.008824777),
   ('next', 0.008520809),
   ('overnight', 0.008380113)]),
 (4,
  [('room', 0.06544511),
   ('pool', 0.03690769),
   ('nice', 0.028441172),
   ('bathroom', 0.026723687),
   ('good', 0.021809824),
   ('shower', 0.021199508),
   ('bed', 0.020384813),
   ('water', 0.012956827),
   ('area', 0.012746755),
   ('view', 0.012573381),
   ('hotel', 0.012447834),
   ('swim', 0.01134797),
   ('breakfast', 0.011058314),
   ('floor', 0.01100214),
   ('clean', 0.010560334),
   ('comfort', 0.010475757),
   ('small', 0.01044164),
   ('bath', 0.010433311),
   ('love', 0.0102905175),
   ('like', 0.010199984)]),
 (5,
  [('hotel', 0.06825839),
   ('good', 0.043090336),
   ('great', 0.03878721),
   ('stay', 0.028584825),
   ('room', 0.02814644),
   ('staff', 0.02141343),
   ('walk', 0.020368688),
   ('restaur', 0.020125745),
   ('nice', 0.019407058),
   ('pool', 0.01892175),
   ('food', 0.018044064),
   ('clean', 0.015541809),
   ('close', 0.015532768),
   ('shop', 0.014632125),
   ('breakfast', 0.013460909),
   ('area', 0.013381537),
   ('excel', 0.013063771),
   ('place', 0.012421849),
   ('also', 0.0112637635),
   ('well', 0.011142297)])]

In [8]:
lda5 = [(0,
  [('hotel', 0.06911704),
   ('room', 0.04318725),
   ('great', 0.040856153),
   ('stay', 0.031329904),
   ('good', 0.026788272),
   ('view', 0.026653612),
   ('shop', 0.020756159),
   ('staff', 0.017083755),
   ('breakfast', 0.016935417),
   ('nice', 0.015988665),
   ('mall', 0.0131834755),
   ('excel', 0.012981376),
   ('restaur', 0.012915415),
   ('food', 0.012654273),
   ('well', 0.012416333),
   ('walk', 0.011805787),
   ('clean', 0.010495572),
   ('area', 0.010311405),
   ('comfort', 0.010213382),
   ('also', 0.0094832545)]),
 (1,
  [('club', 0.1610574),
   ('tea', 0.05872025),
   ('drink', 0.051452283),
   ('afternoon', 0.03976836),
   ('cocktail', 0.03691459),
   ('food', 0.02955796),
   ('great', 0.024905043),
   ('even', 0.017410452),
   ('staff', 0.016674459),
   ('enjoy', 0.016487569),
   ('good', 0.015987394),
   ('high', 0.015053509),
   ('snack', 0.014916568),
   ('breakfast', 0.013775884),
   ('love', 0.01057486),
   ('free', 0.0103764245),
   ('excel', 0.010246777),
   ('recommend', 0.010136923),
   ('well', 0.0097751785),
   ('relax', 0.009698872)]),
 (2,
  [('view', 0.063311145),
   ('pool', 0.061264258),
   ('hotel', 0.034756053),
   ('stay', 0.033663236),
   ('bay', 0.031843293),
   ('room', 0.028298771),
   ('night', 0.024440475),
   ('floor', 0.019626247),
   ('one', 0.018778363),
   ('sand', 0.016888732),
   ('garden', 0.01588092),
   ('worth', 0.015262588),
   ('best', 0.014126427),
   ('top', 0.013988034),
   ('th', 0.012875639),
   ('visit', 0.011843767),
   ('go', 0.0110748345),
   ('place', 0.010686505),
   ('get', 0.009497072),
   ('must', 0.00935249)]),
 (3,
  [('food', 0.05297297),
   ('birthday', 0.037353773),
   ('dinner', 0.029897034),
   ('good', 0.026859112),
   ('restaur', 0.026612362),
   ('great', 0.025265194),
   ('cake', 0.019965872),
   ('buffet', 0.01906084),
   ('lunch', 0.014070557),
   ('swiss', 0.013200009),
   ('nice', 0.012626521),
   ('dine', 0.012604298),
   ('us', 0.0124240285),
   ('also', 0.011656548),
   ('n', 0.01144682),
   ('dollar', 0.009914499),
   ('enjoy', 0.009318243),
   ('come', 0.009134032),
   ('back', 0.00884591),
   ('love', 0.008766781)]),
 (4,
  [('staff', 0.049975075),
   ('stay', 0.047470734),
   ('hotel', 0.03487688),
   ('us', 0.028064009),
   ('great', 0.022291422),
   ('help', 0.02212145),
   ('thank', 0.020740565),
   ('time', 0.014512874),
   ('check', 0.013358878),
   ('made', 0.012969681),
   ('excel', 0.012861036),
   ('back', 0.012515571),
   ('make', 0.012468523),
   ('room', 0.011987047),
   ('wonder', 0.011073718),
   ('special', 0.010790869),
   ('alway', 0.010267705),
   ('would', 0.010250493),
   ('best', 0.010133917),
   ('team', 0.009807765)]),
 (5,
  [('room', 0.046639957),
   ('hotel', 0.02712515),
   ('check', 0.025549129),
   ('stay', 0.012131816),
   ('time', 0.011137077),
   ('get', 0.010378001),
   ('one', 0.009458406),
   ('book', 0.008611085),
   ('would', 0.008262602),
   ('us', 0.008069879),
   ('like', 0.0074813385),
   ('even', 0.007216199),
   ('day', 0.007116313),
   ('guest', 0.0070908577),
   ('bed', 0.007062755),
   ('night', 0.0068365405),
   ('wait', 0.006690296),
   ('staff', 0.006521332),
   ('floor', 0.006488643),
   ('need', 0.006187815)])]

# Dependency Parsing

In [9]:
import spacy
# Pass in LDA topics output to remove overlap words by choosing highest prob
def restruct_topics(topics): 
    word_prob_dict = {}
    for i in range(len(topics)):
        topic_num = topics[i][0]
        for word, prob in topics[i][1]:
            if word in word_prob_dict:
                word_prob_dict[word].append((topic_num, prob))
            else:
                ls = [(topic_num, prob)]
                word_prob_dict[word] = ls
    new_dict = {}
    for word in word_prob_dict:
        topic, highest_prob = max(word_prob_dict[word], key = lambda x: x[1])
        if topic in new_dict:
            new_dict[topic].append(word)
        else:
            ls = [word]
            new_dict[topic] = ls
    return new_dict

In [10]:
nlp = spacy.load("en_core_web_sm")

In [14]:
#takes in the dataframe and appends to each row (aspect, review) pair(s)
def dep_parse(data, new_dict):
    data['aspect_sentiment'] = np.empty((len(data), 0)).tolist()
    for i in range(len(data)):
        #print(i)
        sentence = data['combined_review'][i]
        doc = nlp(sentence)
        aspect_sentiment = []
        for word in doc:
            cond = False
            for x in new_dict.values():
                if word.text in x:
                    cond = True
                    break
            if not cond:
                continue
            if word.pos_ == 'NOUN': 
                for j in word.lefts:
                    #print(j, word, j.dep_, j.pos_)
                    if j.dep_ == 'amod' and j.pos_ == 'ADJ':
                        tup = [word, j, list(new_dict.values()).index(x)]
                        #print(tup)
                        aspect_sentiment.append(tup)
                    for k in j.lefts:
                        if k.dep_ == 'advmod':
                            #print(word, j ,k)
                            tup = [word, k.text + ' ' + j.text, list(new_dict.values()).index(x)]
                            aspect_sentiment.append(tup)
                            if (word, j) in aspect_sentiment:
                                aspect_sentiment.remove([word, j, list(new_dict.values()).index(x)])
        data['aspect_sentiment'][i] = aspect_sentiment

In [24]:
df3 = df3.reset_index(drop=True)
df4 = df4.reset_index(drop=True)
df5 = df5.reset_index(drop=True)

In [29]:
dep_parse(df3, restruct_topics(lda3))

In [30]:
dep_parse(df4, restruct_topics(lda4))

In [31]:
dep_parse(df5, restruct_topics(lda5))

# Textblob

In [32]:
from textblob import TextBlob

## Mean Textblob Function

In [33]:
# Create a function to calculate mean TextBlob polarity for each topic
def calculate_mean_textblob_polarity(row):
    scores = {}
    counts = {}
    
    for aspect_sentiment in row:
        aspect = aspect_sentiment[0]
        adjective = aspect_sentiment[1]
        topic = aspect_sentiment[2]

        # Concatenate aspect and adjective
        text_to_analyze = f"{aspect} {adjective}"

        # Calculate TextBlob polarity
        polarity = TextBlob(text_to_analyze).sentiment.polarity

        # Accumulate polarity values of each aspect within a topic
        column_name = f'topic_{topic}'
        scores[column_name] = scores.get(column_name, 0) + polarity # add polarity to existing polarity (if have)
        # if a topic occurence is more than once, scores[column_name] will be the summed polarity

        # Count occurrences of each aspect within a topic
        count_column_name = f'count_{column_name}'
        counts[count_column_name] = counts.get(count_column_name, 0) + 1
    
    # Calculate mean polarity for each topic
    for count_column_name, topic_count in counts.items():
        #print(counts.items())
        column_name = count_column_name.replace('count_', '')
        mean_polarity = scores[column_name] / counts[count_column_name]
        scores[column_name] = mean_polarity # replace summed polarity with mean polarity

    return pd.Series({**scores})

In [45]:
columns_to_drop = ["combined_review", "date", "stem_review", "lem_review", "star", "cleaned_review"]

In [46]:
result_df3 = df3['aspect_sentiment'].apply(calculate_mean_textblob_polarity)
result_df3["overall_textblob_polarity"] = df3.apply(lambda row: TextBlob(row["combined_review"]).sentiment.polarity, axis = 1)

# Concatenate the result DataFrame with df_subset
df3_with_scores = pd.concat([df3, result_df3], axis=1)

df3_final = df3_with_scores[df3_with_scores['aspect_sentiment'].apply(lambda x: bool(x))] 
df3_final = df3_final.drop(columns = columns_to_drop)

In [47]:
result_df4 = df4['aspect_sentiment'].apply(calculate_mean_textblob_polarity)
result_df4["overall_textblob_polarity"] = df4.apply(lambda row: TextBlob(row["combined_review"]).sentiment.polarity, axis = 1)

# Concatenate the result DataFrame with df_subset
df4_with_scores = pd.concat([df4, result_df4], axis=1)

df4_final = df4_with_scores[df4_with_scores['aspect_sentiment'].apply(lambda x: bool(x))]
print(df4_with_scores.shape)
prin
df4_final = df4_final.drop(columns = columns_to_drop)

In [48]:
result_df5 = df5['aspect_sentiment'].apply(calculate_mean_textblob_polarity)
result_df5["overall_textblob_polarity"] = df5.apply(lambda row: TextBlob(row["combined_review"]).sentiment.polarity, axis = 1)

# Concatenate the result DataFrame with df_subset
df5_with_scores = pd.concat([df5, result_df5], axis=1)

df5_final = df5_with_scores[df5_with_scores['aspect_sentiment'].apply(lambda x: bool(x))]
df5_final = df5_final.drop(columns = columns_to_drop)

In [49]:
csv_name3 = "data/textblob/dp_textblob_3_star.csv"
df3_final.to_csv(here(csv_name3), index=False)

csv_name4 = "data/textblob/dp_textblob_4_star.csv"
df4_final.to_csv(here(csv_name4), index=False)

csv_name5 = "data/textblob/dp_textblob_5_star.csv"
df5_final.to_csv(here(csv_name5), index=False)